In [10]:
import subprocess as sp
import os
import shutil
import time
import multiprocessing as mp
from glob import glob
import pandas as pd

In [24]:
re_min = 1000000
re_max = 8000005
num_core = 4

def create_run_files(params_list, pbs=False):
    cur_dir = params_list[2]
    move_dir = params_list[3]
    af_loc = params_list[4]
    Re = params_list[5]
    af_name = params_list[6]
    alpha_min = params_list[7]
    alpha_max = params_list[8]
    alpha_increment = params_list[9]

    with open(move_dir+af_name+'_{}.run'.format(Re), 'w') as fid2:
        fid2.write('load ' + af_name + '.dat\n')
        #fid2.write('save {}.af\nY\n'.format(af_name))
        fid2.write('pane\n\n')
        fid2.write('plop\n')
        fid2.write('g\n\noper\niter 100\n')
        fid2.write('visc {}\n'.format(str(Re)))
        fid2.write('pacc\n\n\n')
        fid2.write('aseq {} {} {}\n'.format(alpha_min, alpha_max, alpha_increment))
        fid2.write('pacc\npwrt\n')
        fid2.write(af_name + '_Re_' + str(Re) + '.polar\n\n\nquit\n')
    
    if pbs:
        with open(move_dir+af_name+'_{}_pbs.cmd'.format(Re), 'w') as fid2:
            pbs_name = 'run_' + af_name
            pbs_cur_dir = 'kourosh/xf/runs/{}/'.format(af_name)
            pbs_cmd = 'xfoil < {}'.format(af_name+'_{}.run'.format(Re))
            fid2.write(pbs.format(pbs_name, pbs_cur_dir, pbs_cmd))
    
    return

def create_data_structure(Re=[400000, 1000000, 50000],
                          alpha_range=[-10, 35, 1.], af_loc=''):
    ind = 0
    data_structure = []
    cur_dir = os.getcwd() + '/'
    Re = range(Re[0], Re[1], Re[2])
    if af_loc == '':
        af_loc = cur_dir+'af/'
    
    
    # files = glob(af_loc + '*.af')
    files = glob(af_loc + '*.dat')
    
    # names = [i.replace(af_loc, '')[:-3] for i in files]
    names = [i.replace(af_loc, '')[:-4] for i in files]
    
    if not os.path.isdir(cur_dir + 'runs/'):
        os.mkdir(cur_dir + '/runs/')
    
    for i in names:
        
        if not os.path.isdir(cur_dir + 'runs/{}/'.format(i)):
            os.mkdir(cur_dir + '/runs/{}/'.format(i))
            
        xfoil_address = cur_dir + '{}/'.format(i)
        
            
        for k in Re:
            data_structure.append([i, 
                                   cur_dir, 
                                   cur_dir + 'runs/{}/'.format(i),
                                   af_loc, k, i.replace(af_loc, ''),
                                   alpha_range[0], alpha_range[1], alpha_range[2], 0])
        ind += 1
    return data_structure

def move_files(af_loc='/home/ariya/Desktop/github/Airfoil_Polar_prediction/CST/test_analyser/af/', 
               move_dir='/home/ariya/Desktop/github/Airfoil_Polar_prediction/CST/test_analyser/',
               airfoils=True, plts=False):
    if airfoils:
        afs = glob(af_loc+'/*.dat')
        for i in afs:
            af_name = i.replace(af_loc, '')[:-4]
            shutil.copy(i, move_dir + '/runs/{}/'.format(af_name))
    if plts:
        plots = glob(af_loc+'plot/*.png')
        for i in plots:
            af_name = i.replace(af_loc + 'plot/', '')[:-4]
            shutil.copy(i, move_dir + '/runs/{}/'.format(af_name))
    return 

In [28]:
# create the files
# copy the airfoils to the code directory called af the run below
core_num = num_core
data = create_data_structure(Re=[re_min, re_max, 50000])
df = pd.DataFrame(data)
df.columns=['i','cur_dir', 'run_dir', 'af_loc', 'Re', 'af', 'alpha_min', 'alpha_max', 'alpha_step', 'status']
#df['i'] = df['i'].astype(int)
#df.sort_values('i', inplace=True)
df = df.reset_index()
df.to_csv('runs.txt', index=False)
for i in df.index:
    create_run_files(df.iloc[i].values)
move_files(af_loc='/home/ariya/Desktop/github/Airfoil_Polar_prediction/CST/test_analyser/af/', 
           move_dir='/home/ariya/Desktop/github/Airfoil_Polar_prediction/CST/test_analyser/')

['/home/ariya/Desktop/github/Airfoil_Polar_prediction/CST/test_analyser/af/goe279.dat', '/home/ariya/Desktop/github/Airfoil_Polar_prediction/CST/test_analyser/af/mh121.dat', '/home/ariya/Desktop/github/Airfoil_Polar_prediction/CST/test_analyser/af/ag45c03.dat', '/home/ariya/Desktop/github/Airfoil_Polar_prediction/CST/test_analyser/af/usa27m2.dat', '/home/ariya/Desktop/github/Airfoil_Polar_prediction/CST/test_analyser/af/goe330.dat', '/home/ariya/Desktop/github/Airfoil_Polar_prediction/CST/test_analyser/af/c5d.dat', '/home/ariya/Desktop/github/Airfoil_Polar_prediction/CST/test_analyser/af/fxlv152.dat', '/home/ariya/Desktop/github/Airfoil_Polar_prediction/CST/test_analyser/af/fx75141.dat', '/home/ariya/Desktop/github/Airfoil_Polar_prediction/CST/test_analyser/af/b707c.dat', '/home/ariya/Desktop/github/Airfoil_Polar_prediction/CST/test_analyser/af/m23.dat', '/home/ariya/Desktop/github/Airfoil_Polar_prediction/CST/test_analyser/af/e216.dat', '/home/ariya/Desktop/github/Airfoil_Polar_predic